&nbsp;

# 21 - Implémentation du modèle SINDy

---

&nbsp;

> ### Qu'est-ce que la méthode SINDy et comment l'implémenter pour notre jeu de donnée ?

La méthode SINDy (ou *Sparse Identification of Non-linear Dynamic*) cherche à représenter l'évolution temporelle d'un état $x(t)$ sous la forme d'une fonction non-linéaire $f$, soit :

$$
\frac{d}{dt}x(t) = f(x(t))
$$

Exactement la forme que l'on cherche à approximer. On admet que cette équation consistue un système dynamique pour mesurer $x(t)$, où l'état est en fait un vecteur comme suit :

$$
x(t) = [x_1(t), x_2(t). \dots, x_n(t)]^{\perp} 
$$

On peut aisément identifier que $x(t)$ correspond bien à notre état réduit actuel déjà préparé. La fonction $f(x(t))$ vient contraindre comment le système évolue dans le temps. L'idée clé derrière SINDy est que la fonction est souvent répandue dans l'espace d'un ensemble approprié de fonctions de base. On pourrait donner l'exemple suivant :

$$
\frac{d}{dt}x(t) = f(x) = 
\begin{bmatrix}
   f_1(x) \\
   f_2(x)
\end{bmatrix} = 
\begin{bmatrix}
   1 - x_1²x_2 + 8x_1 \\
   1 - 4x_1³
\end{bmatrix}
$$

On dit que la fonction est *parcimonieuse*(ou *sparse*) car la descriptivité est approximée en *sets* de polynômes de deux variables d'une manière que si on écrivait nos fonctions de la forme d'une expansion de fonctions composantes $f_y(x) = \sum^{\infty}_{i=0}\sum^{\infty}_{j=0} a_{i,j}x_1^i x_2^j$, alors il y aurait très peu de coefficients $a_{i,j}$ qui serait non-nuls. 

SINDy emploie une régression parcimonieuse pour chercher une combinaison linéaire de fonctions de base qui capture au mieux la dynamique comporementale du système physique.

In [5]:
# We import all libs we will need to implement properly SINDy method
using NPZ
using LinearAlgebra
using Statistics
using ModelingToolkit
using OrdinaryDiffEq
using Printf
using Statistics
using Plots

# We import our reduced state (build by selection)
data = npzread("data/processed/sstReducedState2COPERNICUS20102019.npz")

# We extract main parameters from it
Z  = Float64.(data["Z"]) #(time, state)
dZ = Float64.(data["dZ"]) # We cast as Float64 to avoid futures errors of type with lib imported functions
splitMask = Int.(data["split"])

# We permut to a preferrable order
Z  = permutedims(Z) #(state, time) 
dZ = permutedims(dZ)

nState, T = size(Z) # Thanks to size(x) function we can extract bounds

# We seek for bounds indexes to delimit test and train sets
# split is a shape of (0, 0, 0, 0, ..., 1, 1, 1)
trainIdx = findall(splitMask .== 0) 
testIdx  = findall(splitMask .== 1)

# We properly define to work easily with our state later
ZTrain  = Z[:, trainIdx]
dZTrain = dZ[:, trainIdx]
ZTest   = Z[:, testIdx]
dZTest  = dZ[:, testIdx];